In [1]:
%load_ext autoreload
%autoreload 2

## Load dataset

In [2]:
import pandas as pd
import numpy as np
import utils.date_formatting

catalog_name = 'whatson_catalog_df'
#featured_movies_df = pd.read_pickle('data/featured_movies_dataframe.pkl')
featured_movies_df = pd.read_parquet('data/whatson_data_fr.parquet')
featured_movies_df.columns

Index(['actors', 'adult', 'available_num_diff', 'collection',
       'consumed_num_diff', 'content_class_key', 'date_diff_1',
       'date_last_diff', 'date_rediff_1', 'date_rediff_2', 'date_rediff_3',
       'date_rediff_4', 'department', 'director', 'duration', 'duration_dt',
       'duration_min', 'end_rights', 'external_reference', 'genres',
       'last_diff_rating', 'last_diff_rating_7', 'missing_tmdb_id',
       'num_diff_RTS1_RTS2', 'num_rights_TV', 'original_language',
       'original_title', 'parental_control', 'popularity', 'processed_title',
       'production_region', 'production_year', 'release_date', 'revenue',
       'start_rights', 'title', 'tmdb_id', 'total_num_diff',
       'valid_num_rights_TV', 'vote_average'],
      dtype='object')

In [3]:
print(f"{len(featured_movies_df)} movies in the dataset")

9990 movies in the dataset


In [4]:
# Catalog consists of all the movies which could potentially be used, thefore drop duplicate movie_ids keeping those in our channel of interest.
# If duplicates in solely competing channels, keep most recent showing between to keep most relevant past audience rating

In [5]:
# Adult = False as default
featured_movies_df['adult'] = np.where(
    featured_movies_df['adult'].isna(),         # condition per-row
    False,                                      # value if True
    featured_movies_df['adult']                 # value if False
)

# Missing original_language put as 'unknown'
featured_movies_df['original_language'] = np.where(
    featured_movies_df['original_language'].isna(),         # condition per-row
    'unknown',                                              # value if True
    featured_movies_df['original_language']                 # value if False
)

featured_movies_df['genres'] = featured_movies_df['genres'].apply(
    lambda x: x if isinstance(x, list) or isinstance(x, np.ndarray) else []
)

# Missing release_date put as '1900-01-01'
featured_movies_df['release_date'] = np.where(
    featured_movies_df['release_date'].isna(),         # condition per-row
    '1900-01-01',                                      # value if True
    featured_movies_df['release_date']                 # value if False
)

# Add a missing_release_date flag
featured_movies_df.loc[:, 'missing_release_date'] = np.where(
    featured_movies_df['release_date'].isna(),  # condition per-row
    False,                                      # value if True
    True                                        # value if False
)
featured_movies_df.loc[:, 'missing_release_date'] = featured_movies_df.loc[:, 'missing_release_date'].apply(lambda s: False if s == '' else True)
featured_movies_df.loc[:, 'release_date'] = featured_movies_df.loc[:, 'release_date'].apply(lambda s: '1900-01-01' if s == '' else s)

# Missing Revenue put as 0 similarly to TMDB API
featured_movies_df['revenue'] = np.where(
    featured_movies_df['revenue'].isna(),         # condition per-row
    0,                                            # value if True
    featured_movies_df['revenue']                 # value if False
)

# missing tmdb id flag
featured_movies_df.loc[:, 'missing_tmdb'] = np.where(
    featured_movies_df['tmdb_id'].isna(),  # condition per-row
    True,                                  # value if True
    False                                  # value if False
)

# Add vote average as zero
featured_movies_df.loc[:, 'vote_average'] = featured_movies_df['vote_average'].fillna(0)

# Add last_diff_rating as zero
featured_movies_df.loc[:, 'last_diff_rating'] = featured_movies_df['last_diff_rating'].fillna(0)

#  Add popularity zero 
featured_movies_df.loc[:, 'popularity'] = featured_movies_df['popularity'].fillna(0)

# Separate Movies and TV Shows
featured_movies_df.loc[:, 'is_movie'] = True

# Age of the movie
dates = pd.to_datetime(featured_movies_df['release_date'])
featured_movies_df.loc[:, 'movie_age'] = (pd.Timestamp.now() - dates).dt.days // 365

In [6]:
featured_movies_df

,actors,adult,available_num_diff,collection,consumed_num_diff,content_class_key,date_diff_1,date_last_diff,date_rediff_1,date_rediff_2,...,start_rights,title,tmdb_id,total_num_diff,valid_num_rights_TV,vote_average,missing_release_date,missing_tmdb,is_movie,movie_age
4,"James Dean, Julie Harris, Raymond Massey, Rich...",False,0.0,A l'est d'éden,1.0,71,1995-05-26,1998-12-24,1998-12-24,None,...,1998-06-01,A l'est d'eden,220.0,1.0,0,7.600,True,False,True,70
59,None,False,2.0,Alice au pays des merveilles,0.0,71,None,None,None,None,...,1995-01-01,Alice au pays des merveilles,12155.0,2.0,0,6.635,True,False,True,15
63,None,False,3.0,Amicalement votre,0.0,71,None,None,None,None,...,1993-01-01,Rocket a monte carlo,NaN,3.0,0,0.000,True,True,True,125
104,"Charles Grodin, Bonnie Hunt, Dean Jones, Tom N...",False,3.0,Beethoven,0.0,71,None,None,None,None,...,1995-11-01,Beethoven,11806.0,3.0,0,5.830,True,False,True,33
115,None,False,2.0,Bonne fête maman,0.0,71,None,None,None,None,...,1992-10-01,Bonne fete maman,279332.0,2.0,0,5.900,True,False,True,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15556,"Dwayne Adway, John Terry, Dee Wallace-Stone",False,1.0,Téléfilm,1.0,71,2002-03-17,2002-03-17,None,None,...,1999-04-01,La vie d'un basketteur hors du commun,NaN,2.0,0,0.000,True,True,True,125
15560,"Laurent Arnal, Marie Desgranges, Philippe Faur...",False,1.0,Téléfilm,1.0,71,1999-03-19,2001-12-08,2001-12-08,None,...,1999-03-01,De gré ou de force,733369.0,2.0,0,5.500,True,False,True,26
16520,"Danica Mckellar, Victor Webster, Bruce Boxleit...",False,0.0,Téléfilm,2.0,71,2024-01-12,2024-08-19,2024-08-19,None,...,2021-09-01,Enquêtes d'amour : Une fiancée trop parfaite,637422.0,2.0,0,6.700,True,False,True,5
17057,"Constance Gay, François-David Cardonnel, Stéph...",False,2.0,Téléfilm,1.0,71,2024-06-07,2024-06-07,None,None,...,2024-03-01,Meurtres à Arles,NaN,3.0,1,0.000,True,True,True,125


In [7]:
# Add times_shown feature
featured_movies_df.loc[:, 'times_shown'] = 0



In [8]:
# Place title as first column
featured_movies_df = featured_movies_df[['title']+featured_movies_df.drop(columns=['title']).columns.tolist()]

In [9]:
drop = ['day', 'doy', 'hour',
       'is_weekend', 'month', 'pdm', 
       'public_holiday', 'rt_m',
       'start_time', 'weekday', 'year', 'season', 'channel'] # Drop temporal context column
for col in drop:
    if col in featured_movies_df.columns:
        print(f"Dropping column: {col}")
        featured_movies_df = featured_movies_df.drop(columns=col)

In [10]:
featured_movies_df[featured_movies_df['valid_num_rights_TV'] > 0]

,title,actors,adult,available_num_diff,collection,consumed_num_diff,content_class_key,date_diff_1,date_last_diff,date_rediff_1,...,start_rights,tmdb_id,total_num_diff,valid_num_rights_TV,vote_average,missing_release_date,missing_tmdb,is_movie,movie_age,times_shown
593,Le voyage de Bashô,None,False,997.0,Documentaire,2.0,71,2021-08-22,2023-06-06,2023-06-06,...,2020-03-08,589953.0,999.0,1,0.000,True,False,True,6,0
837,La rançon,"Patrick Norbert, Katia Rupe, Jean-Pierre Malo,...",False,997.0,Fiction,2.0,71,None,None,None,...,1983-11-09,263118.0,999.0,1,5.210,True,False,True,10,0
934,Frankie et Johnny,"Pfeiffer Michelle, Pacino Al, Hector Elizondo,...",False,1.0,Fiction\Achats,0.0,71,None,None,None,...,2024-08-01,3784.0,1.0,1,6.773,True,False,True,33,0
1035,Vent du large,None,False,999.0,Fiction\Achats,0.0,71,None,None,None,...,1993-09-01,NaN,999.0,1,0.000,True,True,True,125,0
1036,Vent du large,None,False,999.0,Fiction\Achats,0.0,71,None,None,None,...,1993-09-01,NaN,999.0,1,0.000,True,True,True,125,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13097,Le choix de Thomas,"Frédéric Van Den Driessche, Patrick Raynal, Hé...",False,993.0,Louis Page,6.0,71,2004-02-29,2015-05-31,2009-03-13,...,2001-01-01,NaN,999.0,1,0.000,True,True,True,125,0
13638,Loto,"Urs Jucker, Peter Freiburghaus, Esmée Liliane ...",False,998.0,Téléfilm,1.0,71,2020-03-03,2020-03-03,None,...,2016-12-22,211706.0,999.0,1,7.300,True,False,True,12,0
14336,Le dernier des Weynfeldt,"Stefan Kurt, Marie Baumer, Nicholas Ofczarek, ...",False,996.0,Téléfilm,3.0,71,2012-10-21,2021-04-27,2015-10-07,...,2010-08-16,NaN,999.0,1,0.000,True,True,True,125,0
15471,La dynastie Strauss,"Anthony Higgins, Stephen Mcgann, Lisa Harrow, ...",False,999.0,Téléfilm,0.0,71,None,None,None,...,1992-12-01,NaN,999.0,1,0.000,True,True,True,125,0


In [11]:
from utils import date_formatting
from datetime import datetime

rights_cols = ['start_rights', 'end_rights', 'total_num_diff', 'consumed_num_diff', 'available_num_diff',
     'num_diff_RTS1_RTS2', 'num_rights_TV', 'valid_num_rights_TV']
temp = featured_movies_df[date_formatting.to_datetime_format(featured_movies_df['end_rights']) > datetime.now()]
temp[rights_cols]

temp[temp['valid_num_rights_TV'] == 1][rights_cols]




,start_rights,end_rights,total_num_diff,consumed_num_diff,available_num_diff,num_diff_RTS1_RTS2,num_rights_TV,valid_num_rights_TV
593,2020-03-08,2027-03-07,999.0,2.0,997.0,2,1,1
934,2024-08-01,2026-07-31,1.0,0.0,1.0,0,1,1
1195,2025-01-01,2026-12-31,2.0,0.0,2.0,3,3,1
1299,2024-08-01,2026-07-31,1.0,0.0,1.0,3,2,1
1501,2024-08-01,2026-07-31,1.0,0.0,1.0,3,2,1
...,...,...,...,...,...,...,...,...
12680,2024-02-29,2027-02-28,3.0,2.0,1.0,2,1,1
12681,2024-10-31,2027-10-30,3.0,1.0,2.0,1,1,1
13638,2016-12-22,2031-12-21,999.0,1.0,998.0,1,1,1
14336,2010-08-16,2030-08-15,999.0,3.0,996.0,3,1,1


In [12]:
featured_movies_df[featured_movies_df['valid_num_rights_TV'] == 2][rights_cols]

,start_rights,end_rights,total_num_diff,consumed_num_diff,available_num_diff,num_diff_RTS1_RTS2,num_rights_TV,valid_num_rights_TV
7706,2024-01-26,2027-01-25,2.0,1.0,1.0,9,8,2
10546,2019-11-01,2026-10-31,3.0,2.0,1.0,3,3,2
11457,2021-09-01,2025-08-31,2.0,2.0,0.0,2,2,2
11623,2022-09-01,2025-08-31,2.0,2.0,0.0,2,2,2
11707,2023-02-15,2026-02-14,2.0,2.0,0.0,2,2,2


In [13]:
featured_movies_df[featured_movies_df.notna()]['tmdb_id'].value_counts()

tmdb_id
1279104.0    6
172767.0     5
1551.0       5
278927.0     5
321612.0     4
            ..
38167.0      1
44808.0      1
850395.0     1
16409.0      1
637422.0     1
Name: count, Length: 8886, dtype: int64

In [14]:
featured_movies_df[featured_movies_df['tmdb_id'] == 14001.0][['title', 'processed_title']]

,title,processed_title
2658,Dead silence,Dead Silence


In [15]:
featured_movies_df

,title,actors,adult,available_num_diff,collection,consumed_num_diff,content_class_key,date_diff_1,date_last_diff,date_rediff_1,...,start_rights,tmdb_id,total_num_diff,valid_num_rights_TV,vote_average,missing_release_date,missing_tmdb,is_movie,movie_age,times_shown
4,A l'est d'eden,"James Dean, Julie Harris, Raymond Massey, Rich...",False,0.0,A l'est d'éden,1.0,71,1995-05-26,1998-12-24,1998-12-24,...,1998-06-01,220.0,1.0,0,7.600,True,False,True,70,0
59,Alice au pays des merveilles,None,False,2.0,Alice au pays des merveilles,0.0,71,None,None,None,...,1995-01-01,12155.0,2.0,0,6.635,True,False,True,15,0
63,Rocket a monte carlo,None,False,3.0,Amicalement votre,0.0,71,None,None,None,...,1993-01-01,NaN,3.0,0,0.000,True,True,True,125,0
104,Beethoven,"Charles Grodin, Bonnie Hunt, Dean Jones, Tom N...",False,3.0,Beethoven,0.0,71,None,None,None,...,1995-11-01,11806.0,3.0,0,5.830,True,False,True,33,0
115,Bonne fete maman,None,False,2.0,Bonne fête maman,0.0,71,None,None,None,...,1992-10-01,279332.0,2.0,0,5.900,True,False,True,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15556,La vie d'un basketteur hors du commun,"Dwayne Adway, John Terry, Dee Wallace-Stone",False,1.0,Téléfilm,1.0,71,2002-03-17,2002-03-17,None,...,1999-04-01,NaN,2.0,0,0.000,True,True,True,125,0
15560,De gré ou de force,"Laurent Arnal, Marie Desgranges, Philippe Faur...",False,1.0,Téléfilm,1.0,71,1999-03-19,2001-12-08,2001-12-08,...,1999-03-01,733369.0,2.0,0,5.500,True,False,True,26,0
16520,Enquêtes d'amour : Une fiancée trop parfaite,"Danica Mckellar, Victor Webster, Bruce Boxleit...",False,0.0,Téléfilm,2.0,71,2024-01-12,2024-08-19,2024-08-19,...,2021-09-01,637422.0,2.0,0,6.700,True,False,True,5,0
17057,Meurtres à Arles,"Constance Gay, François-David Cardonnel, Stéph...",False,2.0,Téléfilm,1.0,71,2024-06-07,2024-06-07,None,...,2024-03-01,NaN,3.0,1,0.000,True,True,True,125,0


In [16]:
from constants import BASE_CUSTOM_ID
catalog_df = featured_movies_df.drop_duplicates(subset=['processed_title'], keep='first').copy()

# Keep only the rows containing the most information
# Count non-null entries in each row
featured_movies_df['info_score'] = featured_movies_df.notna().sum(axis=1)

# Define a key to identify duplicates — could be 'title', or ['title', 'year'], etc.
group_cols = ['title', 'movie_age']

# Keep row with max info_score within each group
df_dedup = featured_movies_df.sort_values('info_score', ascending=False).drop_duplicates(subset=group_cols)

# Drop helper column
df_dedup = df_dedup.drop(columns='info_score')

# Assign catalog_id
catalog_df['catalog_id'] = catalog_df['tmdb_id'].astype('Int64').astype(str)


# Base ID on TMDB if available, else assign a unique fallback
# We'll start custom IDs from a high value to avoid overlap with real TMDB IDs

# Assign fallback ID for missing tmdb_id
#missing_mask = catalog_df['tmdb_id'].isna()
missing_mask = catalog_df['missing_tmdb'] == True
catalog_df.loc[missing_mask, 'catalog_id'] = [
    BASE_CUSTOM_ID + str(i) for i in range(missing_mask.sum())
    ]

# Convert dates to datetime format
catalog_df['start_rights'] = utils.date_formatting.to_datetime_format(catalog_df['start_rights'])
catalog_df['end_rights'] = utils.date_formatting.to_datetime_format(catalog_df['end_rights'])

catalog_df.reset_index(drop=True, inplace=True)

/var/folders/lx/r6db720s75n16t86hdj6_8qw0000gp/T/ipykernel_46359/730900729.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  featured_movies_df['info_score'] = featured_movies_df.notna().sum(axis=1)


In [17]:
date_cols = ['date_diff_1', 'date_rediff_1', 'date_rediff_2', 'date_rediff_3', 'date_rediff_4', 'date_last_diff']

# now convert each one in place
for col in date_cols:
    catalog_df[col] = pd.to_datetime(
        catalog_df[col],
        errors='coerce',           # invalid parsing → NaT
        infer_datetime_format=True # try to speed up common formats
        # you can also specify dayfirst=True if you have ‘dd/mm/yyyy’ strings
    )

/var/folders/lx/r6db720s75n16t86hdj6_8qw0000gp/T/ipykernel_46359/4248647770.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  catalog_df[col] = pd.to_datetime(
/var/folders/lx/r6db720s75n16t86hdj6_8qw0000gp/T/ipykernel_46359/4248647770.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  catalog_df[col] = pd.to_datetime(
/var/folders/lx/r6db720s75n16t86hdj6_8qw0000gp/T/ipykernel_46359/4248647770.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see http

In [18]:
catalog_df.to_parquet('data/' + catalog_name + '.parquet')

In [19]:
movie_catalog = catalog_df.set_index('catalog_id')

In [20]:
movie_catalog.columns.tolist()

['title',
 'actors',
 'adult',
 'available_num_diff',
 'collection',
 'consumed_num_diff',
 'content_class_key',
 'date_diff_1',
 'date_last_diff',
 'date_rediff_1',
 'date_rediff_2',
 'date_rediff_3',
 'date_rediff_4',
 'department',
 'director',
 'duration',
 'duration_dt',
 'duration_min',
 'end_rights',
 'external_reference',
 'genres',
 'last_diff_rating',
 'last_diff_rating_7',
 'missing_tmdb_id',
 'num_diff_RTS1_RTS2',
 'num_rights_TV',
 'original_language',
 'original_title',
 'parental_control',
 'popularity',
 'processed_title',
 'production_region',
 'production_year',
 'release_date',
 'revenue',
 'start_rights',
 'tmdb_id',
 'total_num_diff',
 'valid_num_rights_TV',
 'vote_average',
 'missing_release_date',
 'missing_tmdb',
 'is_movie',
 'movie_age',
 'times_shown']

In [21]:
movie_catalog

,title,actors,adult,available_num_diff,collection,consumed_num_diff,content_class_key,date_diff_1,date_last_diff,date_rediff_1,...,start_rights,tmdb_id,total_num_diff,valid_num_rights_TV,vote_average,missing_release_date,missing_tmdb,is_movie,movie_age,times_shown
catalog_id,,,,,,,,,,,,,,,,,,,,,
220,A l'est d'eden,"James Dean, Julie Harris, Raymond Massey, Rich...",False,0.0,A l'est d'éden,1.0,71,1995-05-26,1998-12-24,1998-12-24,...,1998-06-01,220.0,1.0,0,7.600,True,False,True,70,0
12155,Alice au pays des merveilles,None,False,2.0,Alice au pays des merveilles,0.0,71,NaT,NaT,NaT,...,1995-01-01,12155.0,2.0,0,6.635,True,False,True,15,0
XF_000_0,Rocket a monte carlo,None,False,3.0,Amicalement votre,0.0,71,NaT,NaT,NaT,...,1993-01-01,NaN,3.0,0,0.000,True,True,True,125,0
11806,Beethoven,"Charles Grodin, Bonnie Hunt, Dean Jones, Tom N...",False,3.0,Beethoven,0.0,71,NaT,NaT,NaT,...,1995-11-01,11806.0,3.0,0,5.830,True,False,True,33,0
279332,Bonne fete maman,None,False,2.0,Bonne fête maman,0.0,71,NaT,NaT,NaT,...,1992-10-01,279332.0,2.0,0,5.900,True,False,True,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XF_000_460,Séparation dangereuse,"Ulrich Tukur, Annett Renneberg, Peter Fitz, Su...",False,1.0,Téléfilm,1.0,71,1999-10-08,2002-03-23,2002-03-23,...,1999-05-01,NaN,2.0,0,0.000,True,True,True,125,0
XF_000_461,La vie d'un basketteur hors du commun,"Dwayne Adway, John Terry, Dee Wallace-Stone",False,1.0,Téléfilm,1.0,71,2002-03-17,2002-03-17,NaT,...,1999-04-01,NaN,2.0,0,0.000,True,True,True,125,0
733369,De gré ou de force,"Laurent Arnal, Marie Desgranges, Philippe Faur...",False,1.0,Téléfilm,1.0,71,1999-03-19,2001-12-08,2001-12-08,...,1999-03-01,733369.0,2.0,0,5.500,True,False,True,26,0


In [22]:
import json
from pathlib import Path

movie_catalog_columns = movie_catalog.columns.tolist()

path = Path("data/constants/movie_catalog_columns.json")
path.parent.mkdir(parents=True, exist_ok=True)

# save
path.write_text(json.dumps(movie_catalog_columns, indent=2))

837